In [307]:
import sys
home_folder = 'C:\\Users\\Nikolaj\\OneDrive - Danmarks Tekniske Universitet\\DTU\\Kandidat\\MasterThesis\\Code\\GoalCornerDetection'
# sys.path.append(r'/zhome/60/1/118435/Master_Thesis/GoalCornerDetection')
sys.path.append(home_folder)
!nvidia-smi

Thu Jan 19 10:32:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.00       Driver Version: 517.00       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8     3W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [308]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
from Core.DataLoader import GoalCalibrationDataset,GoalCalibrationDataset4boxes,GoalCalibrationDatasetOLD,\
train_transform,make_gt_boxes
from torchvision.models.detection import keypointrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from utils import DATA_DIR
import matplotlib.pyplot as plt
from Core.helpers import to_numpy,im_to_numpy,split_data_train_test,test_num_workers, \
find_pixelerror,eval_PCK, PCK_auc, MSE_loss_corners
import numpy as np
import cv2
import json
from Core.plottools import target_to_keypoints, visualize_results, visualize, get_prediction_images, \
crop_image, crop_images, filter_preds, kps_to_cv2, make_pred_image, visualize_cropped_results, visualize_images
import torchvision
from PIL import Image
import albumentations as A # Library for augmentations
import torchvision.transforms as T
from tqdm.notebook import tqdm
from torch.utils.data import random_split,Subset,DataLoader,RandomSampler
from Core.torchhelpers.utils import collate_fn
import pandas as pd
from pprint import pprint
from torchinfo import summary
from torch.utils.data import DataLoader
from Core.torchhelpers.utils import collate_fn
import time
import datetime
from Models.modelzoo import load_custom_batch,loadmymodel
import pickle

In [3]:
## Load datasets
GoalDataold = GoalCalibrationDatasetOLD(DATA_DIR)
GoalData = GoalCalibrationDataset(DATA_DIR)
GoalData4box = GoalCalibrationDataset4boxes(DATA_DIR)

All images: 854
Filtered images: 754
All images: 854
Filtered images: 754
All images: 854
Filtered images: 754
Config not given, using default values for bbox_expand.


In [4]:
train_loader,validation_loader = split_data_train_test(GoalData,
                                                       GoalData,
                                                       validation_split=0.25,
                                                       batch_size=4,
                                                       data_amount=1,
                                                       num_workers=2,
                                                       shuffle_dataset=True,
                                                       shuffle_dataset_seed=20,
                                                       shuffle_epoch = False,
                                                       shuffle_epoch_seed=-1,
                                                       pin_memory=False)

###################
Total size of dataset: 754
Train data --> Size: 566, batch size: 4
Validation data --> Size: 188, batch size: 4


# Script for extracting errors table from wandb

In [27]:
import wandb
root_save_folder = r'C:\\Users\\Nikolaj\\OneDrive - Danmarks Tekniske Universitet\\DTU\\Kandidat\\MasterThesis\\Code\\GoalCornerDetection\\Tests\\saved_data'
run = wandb.init()
artifact_wandbpath = 'nikolaj21/GoalCornerDetection/run-10t4idvg-outliers_table:v0'
artifact = run.use_artifact(artifact_wandbpath, type='run_table')
artifact_dir = artifact.get(root=root_save_folder)
wandb.finish()

wandb: Downloading large artifact run-10t4idvg-outliers_table:v0, 104.66MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.0


TypeError: Data row contained incompatible types:
{'cat': 'all', 'Ndata': 752, 'min': 0.30199143290519714, 'max': 2977.94091796875, 'std': 123.64618682861328, 'mean': 33.12412643432617, 'adj_mean': 24.661212637069376, 'MSE': 16385.583984375, 'median': 12.289907455444336, 'inlier_min': 0.0, 'inlier_max': 404.062686920166, '#outliers': 6, '\\%outliers': 0.007978723404255319, 'outliers': [768.5975952148438, 2977.94091796875, 983.86767578125, 406.0140075683594, 569.05810546875, 658.6328735351562], 'outlier_ids': [116, 128, 426, 658, 678], 'outlier_ims': [{'_type': 'image-file', 'caption': 'Prediction Outlier, Image ID: 116', 'format': 'png', 'height': 3120, 'path': 'media/images/1aa359ccda3d8e5a9cd7.png', 'sha256': 'eaa0de353a0a50fbe9eebed89df40d9adcf0c10bfe1fe2fcae7b49d7b05e1ac4', 'width': 4208}, {'_type': 'image-file', 'caption': 'Prediction Outlier, Image ID: 128', 'format': 'png', 'height': 3120, 'path': 'media/images/15f055a30705b19d061c.png', 'sha256': '5d0fae380d503190bbad0f9a83a66b

In [22]:
print(artifact_dir)
import json
with open(artifact_dir + '/outliers_table.table.json','r') as f:
    table = json.load(f)

C:\\Users\\Nikolaj\\OneDrive - Danmarks Tekniske Universitet\\DTU\\Kandidat\\MasterThesis\\Code\\GoalCornerDetection\\Tests\\saved_data


In [78]:
df = pd.DataFrame(data=table['data'], columns=table['columns'])
df

,cat,Ndata,min,max,std,mean,adj_mean,MSE,median,inlier_min,...,#outliers,\%outliers,outliers,outlier_ids,outlier_ims,outlier_labels,#missing_corners,missing_ids,missing_ims,missing_labels
0,all,752,0.301991,2977.940918,123.646187,33.124126,24.661213,16385.583984,12.289907,0.0,...,6,0.007979,"[768.5975952148438, 2977.94091796875, 983.8676...","[116, 128, 426, 658, 678]","[{'_type': 'image-file', 'caption': 'Predictio...","[1, 1, 1, 1, 1, 1]",0,[],[],[]
1,TL,188,0.301991,75.878731,7.886517,6.814608,5.830881,108.636032,4.702724,0.0,...,4,0.021277,"[75.87873077392578, 31.6424617767334, 43.17889...","[116, 128, 205, 634]","[{'_type': 'image-file', 'caption': 'Predictio...","[1, 1, 1, 1]",0,[],[],[]
2,TR,188,0.460173,2977.940918,218.012161,25.304005,9.463893,48169.597656,5.956155,0.0,...,1,0.005319,[2977.94091796875],[678],"[{'_type': 'image-file', 'caption': 'Predictio...",[1],0,[],[],[]
3,BL,188,0.878046,983.867676,106.115959,56.113197,40.261302,14409.290039,44.149940,0.0,...,4,0.021277,"[768.5975952148438, 983.86767578125, 569.05810...","[116, 128, 426, 658]","[{'_type': 'image-file', 'caption': 'Predictio...","[1, 1, 1, 1]",0,[],[],[]
4,BR,188,0.677536,337.139648,29.924057,44.264690,42.471394,2854.811279,44.206017,0.0,...,1,0.005319,[337.1396484375],[116],"[{'_type': 'image-file', 'caption': 'Predictio...",[1],0,[],[],[]


In [289]:
dfdata = [row[:9] for row in table['data']]
dfcols = table['columns'][:9]
df_report = pd.DataFrame(data=dfdata, columns=dfcols)
df_report

,cat,Ndata,min,max,std,mean,adj_mean,MSE,median
0,all,752,0.301991,2977.940918,123.646187,33.124126,24.661213,16385.583984,12.289907
1,TL,188,0.301991,75.878731,7.886517,6.814608,5.830881,108.636032,4.702724
2,TR,188,0.460173,2977.940918,218.012161,25.304005,9.463893,48169.597656,5.956155
3,BL,188,0.878046,983.867676,106.115959,56.113197,40.261302,14409.290039,44.149940
4,BR,188,0.677536,337.139648,29.924057,44.264690,42.471394,2854.811279,44.206017


In [300]:
print(df_report.apply(go).to_latex(index=False))
print(df_report.apply(go).style.to_latex(hrules=True))

\begin{tabular}{lrrrrrrrr}
\toprule
cat &  Ndata &  min &  max &   std &  mean &  adj\_mean &   MSE &  median \\
\midrule
all &    752 & 0.30 & 2978 & 124.0 &  33.0 &      25.0 & 16386 &    12.0 \\
 TL &    188 & 0.30 &   76 &   7.9 &   6.8 &       5.8 &   109 &     4.7 \\
 TR &    188 & 0.46 & 2978 & 218.0 &  25.0 &       9.5 & 48170 &     6.0 \\
 BL &    188 & 0.88 &  984 & 106.0 &  56.0 &      40.0 & 14409 &    44.0 \\
 BR &    188 & 0.68 &  337 &  30.0 &  44.0 &      42.0 &  2855 &    44.0 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrrrrrrr}
\toprule
 & cat & Ndata & min & max & std & mean & adj_mean & MSE & median \\
\midrule
0 & all & 752 & 0.300000 & 2978 & 124.000000 & 33.000000 & 25.000000 & 16386 & 12.000000 \\
1 & TL & 188 & 0.300000 & 76 & 7.900000 & 6.800000 & 5.800000 & 109 & 4.700000 \\
2 & TR & 188 & 0.460000 & 2978 & 218.000000 & 25.000000 & 9.500000 & 48170 & 6.000000 \\
3 & BL & 188 & 0.880000 & 984 & 106.000000 & 56.000000 & 40.000000 & 14409 & 44.000000 \\
4 &

C:\Users\Nikolaj\AppData\Local\Temp\ipykernel_40552\4210289573.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_report.apply(go).to_latex(index=False))


In [288]:
dfdata = [row[:9]+row[10:13] for row in table['data']]
dfcols = table['columns'][:9] + table['columns'][10:13]
df_report = pd.DataFrame(data=dfdata, columns=dfcols)
df_report

,cat,Ndata,min,max,std,mean,adj_mean,MSE,median,inlier_max,#outliers
0,all,752,0.301991,2977.940918,123.646187,33.124126,24.661213,16385.583984,12.289907,404.062687,6
1,TL,188,0.301991,75.878731,7.886517,6.814608,5.830881,108.636032,4.702724,30.474159,4
2,TR,188,0.460173,2977.940918,218.012161,25.304005,9.463893,48169.597656,5.956155,679.340488,1
3,BL,188,0.878046,983.867676,106.115959,56.113197,40.261302,14409.290039,44.149940,374.461075,4
4,BR,188,0.677536,337.139648,29.924057,44.264690,42.471394,2854.811279,44.206017,134.036861,1


In [283]:
def round_int(a):
#     print(a,type(a))
    if a < 10:
        if a < 1:
            return np.round(a,2)
        else:
#         print('here')
            return np.round(a,1)
    else:
#         print('ugh')
        return int(np.round(a,0))
def go(a):
#     print(a,type(a))
    if pd.to_numeric(a, errors='coerce').notnull().all():
        return a.apply(round_int)
    else:
        return a
df_report.apply(go)

,cat,Ndata,min,max,std,mean,adj_mean,MSE,median,inlier_max,#outliers,\%outliers
0,all,752,0.30,2978,124.0,33.0,25.0,16386,12.0,404,6,0.01
1,TL,188,0.30,76,7.9,6.8,5.8,109,4.7,30,4,0.02
2,TR,188,0.46,2978,218.0,25.0,9.5,48170,6.0,679,1,0.01
3,BL,188,0.88,984,106.0,56.0,40.0,14409,44.0,374,4,0.02
4,BR,188,0.68,337,30.0,44.0,42.0,2855,44.0,134,1,0.01


In [286]:
round_cols = ['Ndata','min','max','std','mean','adj_mean','MSE','median','inlier_max','#outliers']

for col in round_cols:
    df_report[col] = df_report[col].apply(round_int)
df_report

,cat,Ndata,min,max,std,mean,adj_mean,MSE,median,inlier_max,#outliers,\%outliers
0,all,752,0.30,2978,124.0,33.0,25.0,16386,12.0,404,6,0.01
1,TL,188,0.30,76,7.9,6.8,5.8,109,4.7,30,4,0.02
2,TR,188,0.46,2978,218.0,25.0,9.5,48170,6.0,679,1,0.01
3,BL,188,0.88,984,106.0,56.0,40.0,14409,44.0,374,4,0.02
4,BR,188,0.68,337,30.0,44.0,42.0,2855,44.0,134,1,0.01


In [285]:
print(df_report.style.to_latex())

\begin{tabular}{llrrrrrrrrrrr}
 & cat & Ndata & min & max & std & mean & adj_mean & MSE & median & inlier_max & #outliers & \%outliers \\
0 & all & 752 & 0.300000 & 2978 & 124.000000 & 33.000000 & 25.000000 & 16386 & 12.000000 & 404 & 6 & 0.010000 \\
1 & TL & 188 & 0.300000 & 76 & 7.900000 & 6.800000 & 5.800000 & 109 & 4.700000 & 30 & 4 & 0.020000 \\
2 & TR & 188 & 0.460000 & 2978 & 218.000000 & 25.000000 & 9.500000 & 48170 & 6.000000 & 679 & 1 & 0.010000 \\
3 & BL & 188 & 0.880000 & 984 & 106.000000 & 56.000000 & 40.000000 & 14409 & 44.000000 & 374 & 4 & 0.020000 \\
4 & BR & 188 & 0.680000 & 337 & 30.000000 & 44.000000 & 42.000000 & 2855 & 44.000000 & 134 & 1 & 0.010000 \\
\end{tabular}



# Extract prediction images from a model